
# Meta-Analysis: Synthesizing Evidence from Multiple Studies

This notebook introduces meta-analysis techniques, including fixed and random effects models, meta-regression, and visualization methods like forest plots and funnel plots.

## Dataset Information and Links

1. **Meta-analysis Data Sources:**
   - Example datasets for meta-analysis can be found in the `metafor` R package, which provides CSV exports: [Metafor Example Data](https://www.metafor-project.org/doku.php/data/datasets)
   - Alternatively, use any clinical study datasets where outcomes and sample sizes are available.

### Prerequisites

Install the required packages before running the code:
```bash
pip install meta-analysis numpy pandas matplotlib seaborn
```
        


## Fixed Effects Meta-Analysis

Fixed effects models assume that all studies share the same underlying effect size. Let's calculate the pooled effect size using inverse variance weighting.
        

In [ ]:

import numpy as np
import pandas as pd

# Example data: Study effect sizes and variances
data = pd.DataFrame({
    "Study": ["Study 1", "Study 2", "Study 3"],
    "EffectSize": [0.2, 0.5, 0.3],
    "Variance": [0.02, 0.03, 0.01]
})

# Calculate weights (inverse variance)
data["Weight"] = 1 / data["Variance"]

# Pooled effect size
pooled_effect = np.sum(data["EffectSize"] * data["Weight"]) / np.sum(data["Weight"])

# Variance of pooled effect
pooled_variance = 1 / np.sum(data["Weight"])

print(f"Pooled Effect Size (Fixed Effects): {pooled_effect:.3f}")
print(f"Variance of Pooled Effect: {pooled_variance:.3f}")
        


## Random Effects Meta-Analysis

Random effects models account for heterogeneity among studies. We'll use the DerSimonian and Laird method for random effects meta-analysis.
        

In [ ]:

# Estimate between-study variance (tau-squared)
Q = np.sum(data["Weight"] * (data["EffectSize"] - pooled_effect)**2)
df = len(data) - 1  # Degrees of freedom
tau_squared = max(0, (Q - df) / (np.sum(data["Weight"]) - np.sum(data["Weight"]**2) / np.sum(data["Weight"])))

# Adjust weights for random effects
data["RandomWeight"] = 1 / (data["Variance"] + tau_squared)

# Random effects pooled effect size
random_effect = np.sum(data["EffectSize"] * data["RandomWeight"]) / np.sum(data["RandomWeight"])

# Variance of random effects pooled effect
random_variance = 1 / np.sum(data["RandomWeight"])

print(f"Pooled Effect Size (Random Effects): {random_effect:.3f}")
print(f"Variance of Random Effects Pooled Effect: {random_variance:.3f}")
        


## Forest Plot

A forest plot visualizes individual study effects and the overall pooled effect.
        

In [ ]:

import matplotlib.pyplot as plt

# Forest plot
plt.errorbar(data["EffectSize"], range(len(data)), xerr=np.sqrt(data["Variance"]), fmt="o", label="Individual Studies")
plt.axvline(pooled_effect, color="red", linestyle="--", label="Pooled Effect (Fixed)")
plt.axvline(random_effect, color="blue", linestyle="--", label="Pooled Effect (Random)")
plt.yticks(range(len(data)), data["Study"])
plt.xlabel("Effect Size")
plt.title("Forest Plot")
plt.legend()
plt.gca().invert_yaxis()  # Reverse y-axis for readability
plt.show()
        


## Funnel Plot for Publication Bias

A funnel plot assesses publication bias by visualizing effect sizes versus their standard errors.
        

In [ ]:

# Standard errors
data["StdError"] = np.sqrt(data["Variance"])

# Funnel plot
plt.scatter(data["EffectSize"], 1 / data["StdError"], alpha=0.7)
plt.axvline(pooled_effect, color="red", linestyle="--", label="Pooled Effect")
plt.xlabel("Effect Size")
plt.ylabel("Precision (1 / StdError)")
plt.title("Funnel Plot")
plt.legend()
plt.show()
        


## Meta-Regression

Meta-regression examines the relationship between effect sizes and study-level covariates.
        

In [ ]:

# Example covariate data
data["SampleSize"] = [50, 100, 150]

# Regression line
from sklearn.linear_model import LinearRegression

X = data["SampleSize"].values.reshape(-1, 1)
y = data["EffectSize"]

model = LinearRegression().fit(X, y)
trendline = model.predict(X)

# Meta-regression plot
plt.scatter(data["SampleSize"], data["EffectSize"], alpha=0.7, label="Studies")
plt.plot(data["SampleSize"], trendline, color="red", label="Trend Line")
plt.xlabel("Sample Size")
plt.ylabel("Effect Size")
plt.title("Meta-Regression Plot")
plt.legend()
plt.show()
        